In [13]:
import pandas as pd
from pathlib import Path
import os
import re

def get_available_countries(base_path):
    """Get list of country codes from filenames in the directory."""
    pattern = r'v2_power_plant_phaseout_order_by_maturity_([A-Z]{2})_2050.csv'
    country_codes = set()
    
    for file in os.listdir(base_path):
        match = re.match(pattern, file)
        if match:
            country_codes.add(match.group(1))
    
    return sorted(list(country_codes))

def create_ranked_phaseout_excel(base_path, output_path):
    # Get available countries
    countries = get_available_countries(base_path)
    if not countries:
        print("No country files found!")
        return
    
    print(f"Found {len(countries)} countries to process")
    
    with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
        for country_code in countries:
            print(f"\nProcessing country: {country_code}")
            
            # Define criteria files in desired order
            criteria_files = {
                'Maturity': f'v2_power_plant_phaseout_order_by_maturity_{country_code}_2050.csv',
                'Emission Factor': f'v2_power_plant_phaseout_order_by_emission_factor_{country_code}_2050.csv',
                'Benefits/Cost/Maturity': f'v2_power_plant_phaseout_order_by_emissions_per_OC_maturity_{country_code}_2050.csv',
                'Benefits/Cost': f'v2_power_plant_phaseout_order_by_emissions_per_opportunity_cost_projection_{country_code}_2050.csv'
            }
            
            # List to store DataFrames
            dfs = []
            
            # Process each criteria file
            for criteria_name, filename in criteria_files.items():
                file_path = Path(base_path) / filename
                if not os.path.exists(file_path):
                    print(f"File not found: {filename}")
                    continue
                
                # Read CSV and add rank based on index
                df = pd.read_csv(file_path)
                df['rank'] = df.index + 1
                
                # Keep needed columns
                columns_to_keep = ['rank', 'asset_name', 'subsector', 'fraction', 'amount_mtco2', 'year']
                df = df[columns_to_keep]
                
                # Format fractions as percentages
                df['fraction'] = df['fraction'].apply(lambda x: f"{x*100:.1f}%")
                
                # Calculate actual emissions
                df['amount_mtco2'] = (df['amount_mtco2'] * df['fraction'].str.rstrip('%').astype(float) / 100).round(3)
                
                # Rename columns
                rename_dict = {col: f"{col} ({criteria_name})" for col in columns_to_keep}
                df = df.rename(columns=rename_dict)
                
                # Add narrow empty column after each criteria (except the last one)
                if criteria_name != 'Benefits/Cost':
                    df['_'] = ''  # Using underscore for empty column name
                
                dfs.append(df)
            
            # Combine all DataFrames
            result_df = pd.concat(dfs, axis=1)
            
            # Write to Excel
            sheet_name = f"{country_code}"
            result_df.to_excel(writer, sheet_name=sheet_name, index=False)
            
            # Adjust column widths
            worksheet = writer.sheets[sheet_name]
            for idx, col in enumerate(result_df.columns):
                # Set narrow width for separator columns
                if col == '_':
                    width = 2
                else:
                    max_length = max(
                        result_df[col].astype(str).apply(len).max(),
                        len(str(col))
                    )
                    width = max_length + 2
                
                # Convert column index to Excel column letter
                col_letter = chr(65 + idx) if idx < 26 else chr(65 + (idx//26) - 1) + chr(65 + (idx%26))
                worksheet.column_dimensions[col_letter].width = width
    
    print("\nDone!")

if __name__ == "__main__":
    base_path = "/Users/yukthabhadane/Documents/Climate Finance Thesis/Paper Alissa Jan 2025/Phase out data"
    output_path = Path(base_path).parent / "v3_Phase_Out_Rankings_Combined.xlsx"
    df = create_ranked_phaseout_excel(base_path, output_path)


Found 15 countries to process

Processing country: BD

Processing country: CL

Processing country: DO

Processing country: EG

Processing country: HN

Processing country: ID

Processing country: IN

Processing country: IR

Processing country: MX

Processing country: PK

Processing country: PL

Processing country: TH

Processing country: TR

Processing country: VN

Processing country: ZA

Done!


In [1]:
import pandas as pd
from pathlib import Path
import os

def create_asset_centric_excel(base_path, output_path):
    countries = {
        'IN': 'India', 'ID': 'Indonesia', 'ZA': 'South Africa', 
        'MX': 'Mexico', 'VN': 'Vietnam', 'IR': 'Iran', 
        'TH': 'Thailand', 'EG': 'Egypt', 'TR': 'Türkiye', 'BD': 'Bangladesh'}
    
    with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
        for country_code, country_name in countries.items():
            print(f"\nProcessing {country_name}...")
            
            # Read all four criteria files
            dfs = {}
            criteria_files = {
                'maturity': f'v2_power_plant_phaseout_order_by_maturity_{country_code}_2050.csv',
                'emission': f'v2_power_plant_phaseout_order_by_emission_factor_{country_code}_2050.csv',
                'benefits': f'v2_power_plant_phaseout_order_by_emissions_per_opportunity_cost_projection_{country_code}_2050.csv',
                'benefits_maturity': f'v2_power_plant_phaseout_order_by_emissions_per_OC_maturity_{country_code}_2050.csv'
            }
            
            # First, collect all dataframes and add ranks
            for criteria, filename in criteria_files.items():
                file_path = Path(base_path) / filename
                if os.path.exists(file_path):
                    df = pd.read_csv(file_path)
                    df['rank'] = range(1, len(df) + 1)
                    dfs[criteria] = df
            
            if not dfs:  # Skip if no files found for country
                print(f"No data files found for {country_name}")
                continue
                
            # Get unique assets from all dataframes
            all_assets = set()
            for df in dfs.values():
                all_assets.update(df['asset_name'].unique())
            
            # Create consolidated dataframe
            asset_data = []
            
            for asset in all_assets:
                row = {'asset_name': asset}
                
                # Initialize with base data from first available criteria
                for criteria, df in dfs.items():
                    asset_info = df[df['asset_name'] == asset]
                    if not asset_info.empty:
                        row['subsector'] = asset_info.iloc[0]['subsector']
                        row['amount_mtco2'] = asset_info.iloc[0]['amount_mtco2']
                        break
                
                # Get rankings and years for each criteria
                for criteria, df in dfs.items():
                    asset_info = df[df['asset_name'] == asset]
                    if not asset_info.empty:
                        row[f'year_{criteria}'] = asset_info.iloc[0]['year']
                        row[f'rank_{criteria}'] = asset_info.iloc[0]['rank']
                    else:
                        row[f'year_{criteria}'] = None
                        row[f'rank_{criteria}'] = None
                
                asset_data.append(row)
            
            if not asset_data:  # Skip if no assets found
                print(f"No assets found for {country_name}")
                continue
                
            # Create DataFrame and sort by maturity rank
            consolidated_df = pd.DataFrame(asset_data)
            
            # Sort by maturity rank, handling None values
            consolidated_df = consolidated_df.sort_values(
                'rank_maturity', 
                na_position='last'
            )
            
            # Rename columns for clarity
            column_mapping = {
                'year_maturity': 'Year (Maturity)',
                'rank_maturity': 'Rank (Maturity)',
                'year_emission': 'Year (Emission Factor)',
                'rank_emission': 'Rank (Emission Factor)',
                'year_benefits': 'Year (Benefits/Cost)',
                'rank_benefits': 'Rank (Benefits/Cost)',
                'year_benefits_maturity': 'Year (Benefits/Cost/Maturity)',
                'rank_benefits_maturity': 'Rank (Benefits/Cost/Maturity)',
                'amount_mtco2': 'Emissions (MtCO2)',
                'asset_name': 'Asset Name',
                'subsector': 'Subsector'
            }
            consolidated_df = consolidated_df.rename(columns=column_mapping)
            
            # Reorder columns
            column_order = [
                'Asset Name', 'Subsector', 'Emissions (MtCO2)',
                'Year (Maturity)', 'Rank (Maturity)',
                'Year (Emission Factor)', 'Rank (Emission Factor)',
                'Year (Benefits/Cost)', 'Rank (Benefits/Cost)',
                'Year (Benefits/Cost/Maturity)', 'Rank (Benefits/Cost/Maturity)'
            ]
            consolidated_df = consolidated_df[column_order]
            
            # Write to Excel
            sheet_name = f"{country_code}"
            consolidated_df.to_excel(writer, sheet_name=sheet_name, index=False)
            
            # Auto-adjust column widths
            worksheet = writer.sheets[sheet_name]
            for i, col in enumerate(consolidated_df.columns):
                max_length = max(
                    consolidated_df[col].astype(str).apply(len).max(),
                    len(str(col))
                )
                # Add some padding
                worksheet.column_dimensions[chr(65 + i)].width = max_length + 2

if __name__ == "__main__":
    base_path = "/Users/yukthabhadane/Documents/Climate Finance Thesis/Paper Alissa Jan 2025/Phase out data"
    output_path = Path(base_path).parent / "Phase_Out_Rankings_Combined.xlsx"
    
    print("Creating combined Excel file...")
    create_asset_centric_excel(base_path, output_path)
    print("\nDone!")

Creating combined Excel file...

Processing India...

Processing Indonesia...

Processing South Africa...

Processing Mexico...

Processing Vietnam...

Processing Iran...

Processing Thailand...

Processing Egypt...

Processing Türkiye...
No data files found for Türkiye

Processing Bangladesh...
No data files found for Bangladesh

Done!
